In [1]:
import requests
import pattern3.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from progressbar import ProgressBar
from bs4 import BeautifulSoup

<br><br/> 

In [12]:
class build_dataset(object):
    
    def __init__(self, terms=None):
        
        self.names_ = []
        self.genres_ = [] 
        self.genreIDs_ = []
        self.IDs_desc_ = []
        self.terms = terms 
        
        
        


    def pull_genres(self, my_data):
        

        all_genres = [] 
        all_genreIDs = []
        all_genreswithIDs = [] 
        new_list_IDs = []
        urls = [] 
        rss_feeds = [] 
        names = []
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL'] 

        
        pbar = ProgressBar()
        
        
        
        
        if my_data["resultCount"] != 0: 
            for elem in pbar(my_data["results"]):   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                        
                        if "trackCount" in elem.keys():
                            if elem["trackCount"] >= 20: 
                                
                                episode_count = elem["trackCount"]
                                
                                if "genreIds" in elem.keys():
                                    genreID = elem["genreIds"]
                                    all_genreIDs.append(genreID)

                                if "genres" in elem.keys():
                                    genre = elem["genres"] 
                                    all_genres.append(genre)
                                    genresandIDs = list(zip(genreID, genre))
                                    all_genreswithIDs.append(genresandIDs)

                                if "collectionName" in elem.keys():
                                    collection_name = elem["collectionName"]
                                    names.append(collection_name)


                                if "collectionViewUrl" in elem.keys(): 
                                    itunes_url = elem["collectionViewUrl"]
                                                                                           
                                elif "trackViewUrl" in elem.keys():
                                    itunes_url = elem["trackViewUrl"]
                                    

                                if "feedUrl" in elem.keys():
                                    feedurl = elem["feedUrl"] 
                                    
                                    
                                if "artworkUrl100" in elem.keys():
                                    artwork = elem["artworkUrl100"]
                                    
                                


                                my_row = [collection_name, artwork, episode_count, genreID, itunes_url, feedurl]
                                my_row_pd = pd.DataFrame([my_row], columns = columns)                    
                                self.df_podcast_ = self.df_podcast_.append(my_row_pd, ignore_index=True)



                                self.genres_ = list(set(np.concatenate(all_genres).tolist()))
                                self.genreIDs_ = list(set(np.concatenate(all_genreIDs).tolist()))
                                self.names_ = names  
                                new_list_IDs = np.concatenate(all_genreswithIDs).tolist()
                                self.IDs_desc_ = [ii for n,ii in enumerate(new_list_IDs) if ii not in new_list_IDs[:n]]
                        
                        else:
                            pass 


                    else:
                        pass 

                else:
                    pass 
                   
        else: 
            pass 
        
        return self
        

    
    def build_random_data(self, term):
        
        
        url = "https://itunes.apple.com/search?term=" + term + "&limit=200&country=PH&lan=en_us&entity=podcast"
        
        try:
            response = requests.get(url) 
            data = response.json()

            print (data["resultCount"])
            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 



    def build_genres_data(self, genre):
        
        url = "https://itunes.apple.com/search?term=" + genre + "&limit=200&country=PH&lan=en_us&entity=podcast"
        
        try: 
            response = requests.get(url) 
            data = response.json()

            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 


        
        
    def find_genres(self, gnr):
        
        self.build_genres_data(gnr)

        genres_list = self.genres_
        names_list = self.names_
        IDs_genres_list = self.IDs_desc_

        if names_list != []: 
            for name in names_list:
                if name not in self.initial_names_:
                    self.initial_names_.append(name)
        
        if IDs_genres_list != []:
            for item in IDs_genres_list:
                if item not in self.genres_with_IDs_:
                    self.genres_with_IDs_.append(item)

        if genres_list != []: 
            for genre in genres_list:
                if genre not in self.initial_genres_:
                    self.initial_genres_.append(genre)
                    print ("extra")
                    self.find_genres(genre)
                  
        return self 
                    
        

    def initialize(self):
        self.initial_genres_ = None
        self.genreIDs_ = None
        self.names_ = None 
        self.genres_with_IDs_ = None
        
        url = "https://itunes.apple.com/search?term=podcasts&limit=200&country=PH&lan=en_us&entity=podcast" 
        
        
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL']
        self.df_podcast_ = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns = columns)

        try: 
            response = requests.get(url) 
            my_data = response.json()

            if my_data["resultCount"] != 0:
                self.pull_genres(my_data)

                self.initial_genres_ = self.genres_
                self.initial_genreIDs_ = self.genreIDs_
                self.genres_with_IDs_ = self.IDs_desc_
                self.initial_names_ = self.names_
                

            else: 
                pass 
            
        except ValueError:
            print ("Decoding JSON has failed")
        

                    
    def build_data(self):
        
        terms = self.terms
        self.initialize()
        
        if self.initial_genres_ is not None:
            for initial_genre in self.initial_genres_: 
                self.find_genres(initial_genre)
        
        print (terms)
        if terms is not None:
            for term in terms:
                self.build_random_data(term)
        

        return self 
        

In [13]:
more_genres = ["random", "editorial", "review", "hobbies", "fiction", "mystery", "adventure", "romance", "humor"]

In [14]:
data_builder = build_dataset(more_genres)

In [15]:
data_builder.build_data()

100% |########################################################################|
100% |########################################################################|


extra
extra


100% |########################################################################|


extra
extra


100% |########################################################################|


...

extra
extra
extra
extra


100% |########################################################################|


extra


extra


UnboundLocalError: local variable 'feedurl' referenced before assignment

In [16]:
genre_IDs = data_builder.genres_with_IDs_

with open('genre_IDs.txt','a') as t:
    t.write('\n' + str(genre_IDs))      
    t.close()

In [17]:
podcast_dataframe = data_builder.df_podcast_.drop_duplicates(['Name']).reset_index(drop=True)

In [18]:
podcast_dataframe.drop(podcast_dataframe.index[0], inplace=True)

In [19]:
podcast_dataframe = podcast_dataframe.reset_index(drop=True)

In [20]:
podcast_dataframe.to_csv("df_01.csv", index=False)

In [21]:
podcast_dataframe

,Name,Artwork,Episode Count,GenreIDs,iTunes URL,FeedURL
0,Barangay Love Stories,https://is1-ssl.mzstatic.com/image/thumb/Podca...,173,"[1324, 26]",https://podcasts.apple.com/ph/podcast/barangay...,https://www.gmanetwork.com/radio/podcast/10000
1,TED Talks Daily,https://is5-ssl.mzstatic.com/image/thumb/Podca...,275,"[1402, 26, 1301, 1304]",https://podcasts.apple.com/ph/podcast/ted-talk...,http://feeds.feedburner.com/TEDTalks_audio
2,Stuff You Missed in History Class,https://is4-ssl.mzstatic.com/image/thumb/Podca...,300,"[1487, 26, 1324]",https://podcasts.apple.com/ph/podcast/stuff-yo...,https://feeds.megaphone.fm/stuffyoumissedinhis...
3,History Extra podcast,https://is4-ssl.mzstatic.com/image/thumb/Podca...,300,"[1487, 26, 1324]",https://podcasts.apple.com/ph/podcast/history-...,http://rss.acast.com/historyextra
4,Monster Radio RX93.1's Official Podcast Channel,https://is4-ssl.mzstatic.com/image/thumb/Featu...,298,"[1310, 26]",https://podcasts.apple.com/ph/podcast/monster-...,http://www.rx931.com/rxpodcasts.xml
...,...,...,...,...,...,...
1180,Holistic Pet Radio,https://is4-ssl.mzstatic.com/image/thumb/Podca...,64,"[1304, 26, 1512, 1513]",https://podcasts.apple.com/ph/podcast/holistic...,http://www.spreaker.com/show/1906839/episodes/...
1181,Raising Your Paws- Your resource for dog & cat...,https://is3-ssl.mzstatic.com/image/thumb/Podca...,50,"[1305, 26, 1304]",https://podcasts.apple.com/ph/podcast/raising-...,https://feeds.blubrry.com/feeds/raisingyourpaw...
1182,The Pet Photographers Club,https://is3-ssl.mzstatic.com/image/thumb/Podca...,46,"[1321, 26, 1301, 1406]",https://podcasts.apple.com/ph/podcast/the-pet-...,https://thepetphotographersclub.squarespace.co...
1183,Nine Lives with Dr. Kat - Cat podcasts for cat...,https://is5-ssl.mzstatic.com/image/thumb/Podca...,55,"[1305, 26, 1304]",https://podcasts.apple.com/ph/podcast/nine-liv...,http://feeds.feedburner.com/NineLivesWithDrKat...
